## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [219]:
import pandas as pd
import numpy as np
import math as math

from google.colab import drive 
drive.mount('/content/drive')

In [333]:
tr_mcc_codes_path = '/content/drive/MyDrive/Data/tr_mcc_codes.csv'
tr_types_path = '/content/drive/MyDrive/Data/tr_types.csv'
gender_train_path = '/content/drive/MyDrive/Data/gender_train.csv'
transaction_path = '/content/drive/MyDrive/Data/transactions.csv'

tr_mcc_codes_df = pd.read_csv(tr_mcc_codes_path, sep = ';')
tr_types_df = pd.read_csv(tr_types_path, sep = ";")
gender_train_df = pd.read_csv(gender_train_path, sep = ",")
transaction_reader = pd.read_csv(transaction_path, sep = ",", iterator=True, chunksize=1000000)
transaction_df = transaction_reader.get_chunk(1000000)


###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [295]:
randomTransactions = transaction_df['tr_type'].sample(n = 1000)
type(randomTransactions)
cntPOSorATM = 0

for index, value in randomTransactions.items():
  if tr_types_df.query("tr_type=={}".format(value))['tr_description'].str.contains("ATM|POS").all():
    cntPOSorATM += 1

print(cntPOSorATM / 1000)




0.368


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [318]:
frequency = transaction_df['tr_type'].value_counts().to_dict()
top10 = {}
top = 10

for key in frequency:
  if top > 0:
    top10[key] = frequency[key]
    print(key, ' ', tr_types_df.query("tr_type=={}".format(key))['tr_description'])
    top -= 1


1010   98    Покупка. POS ТУ СБ РФ
Name: tr_description, dtype: object
1110   101    Покупка. POS ТУ Россия
Name: tr_description, dtype: object
2010   106    Выдача наличных в АТМ Сбербанк России
Name: tr_description, dtype: object
1030   99    Оплата услуги. Банкоматы СБ РФ
Name: tr_description, dtype: object
2370   129    Списание с карты на карту по операции <перевод...
Name: tr_description, dtype: object
7070   59    Перевод на карту (с карты) через Мобильный бан...
Name: tr_description, dtype: object
7010   39    Взнос наличных через АТМ (в своем тер.банке)
Name: tr_description, dtype: object
1100   100    Покупка. ТУ  Россия
Name: tr_description, dtype: object
1210   103    Покупка. POS  Зарубеж. ТУ
Name: tr_description, dtype: object
1200   102    Покупка. Зарубеж. ТУ
Name: tr_description, dtype: object


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [324]:
max_value = transaction_df["amount"].max()
min_value = transaction_df["amount"].min()
# print(max_value, min_value)

max_amount_customer_id = transaction_df.query("amount=={}".format(max_value))['customer_id'].unique()
min_amount_customer_id = transaction_df.query("amount=={}".format(min_value))['customer_id'].unique()

print(max_amount_customer_id, min_amount_customer_id)

diff = max_value - min_value
print(diff * ((diff > 0) - (diff < 0)))

[19950741] [19950741]
269509892.62


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [329]:
sumForTop10FrequentTransactions = 0
medianForTop10FrequentTransactions = 0
cnt = -1

for key in top10:
  cnt += 1
  if (cnt == 4): medianForTop10FrequentTransactions += transaction_df.query("tr_type=={}".format(key))['amount'].sum()
  if (cnt == 5): 
    medianForTop10FrequentTransactions += transaction_df.query("tr_type=={}".format(key))['amount'].sum()
    medianForTop10FrequentTransactions /= 2

  sumForTop10FrequentTransactions += transaction_df.query("tr_type=={}".format(key))['amount'].mean()

averageForTop10FrequentTransactions = sumForTop10FrequentTransactions / 10;
#4.1
print(averageForTop10FrequentTransactions, medianForTop10FrequentTransactions)

averageForClientWithMaxAmount = transaction_df.query("customer_id=={}".format(max_amount_customer_id))['amount'].mean()
averageForClientWithMinAmount = transaction_df.query("customer_id=={}".format(min_amount_customer_id))['amount'].mean()

medianForClientWithMaxAmount = transaction_df.query("customer_id=={}".format(max_amount_customer_id))['amount'].median()
medianForClientWithMinAmount = transaction_df.query("customer_id=={}".format(min_amount_customer_id))['amount'].median()

#4.2
print(averageForClientWithMaxAmount, averageForClientWithMinAmount, medianForClientWithMaxAmount, medianForClientWithMinAmount)


745.2899996035634 -346209342.345
-320771.81621436717 -320771.81621436717 -33600.024999999994 -33600.024999999994


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [339]:
transaction_df = pd.merge(transaction_df, gender_train_df, how='left')
transaction_df = pd.merge(transaction_df, tr_mcc_codes_df, how='inner')
transaction_df = pd.merge(transaction_df, tr_types_df, how='inner')
transaction_df.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [367]:
averageSpendingZeroGender = transaction_df.query('gender == 0 & amount < 0')['amount'].mean()
averageSpendingNonZeroGender = transaction_df.query('gender != 0 & amount < 0')['amount'].mean()
diffNegative = averageSpendingZeroGender - averageSpendingNonZeroGender
#5.1
print(diffNegative * ((diffNegative > 0) - (diffNegative < 0)))

averageEarningZeroGender = transaction_df.query('gender == 0 & amount > 0')['amount'].mean()
averageEarningNonZeroGender = transaction_df.query('gender != 0 & amount > 0')['amount'].mean()
diffPositive = averageEarningZeroGender - averageEarningNonZeroGender
#5.2
print(diffPositive * ((diffPositive > 0) - (diffPositive < 0)))

16302.46190983049
39073.70587002412


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 